In [31]:
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.quantum_info import SparsePauliOp

def ops_from_file(filename, num_lines=False):
    """Read Hamiltonian from file and return list of qiskit.opflow.PrimitiveOp
    File format: 'coefficient Pauli-string' per line, separated by space
    Optional parameter num_lines: only read first # operators
    """
    try:
        file = open(filename) # read file
    except:
        print("An exception occurred trying to read the file: " + filename) 
    lines = file.read().split('\n')
    i = 1
    ret = []
    for l in lines:
        if num_lines and i > num_lines: # reached max. number lines
            return ret
        if len(l): # exclude empty lines
            [c,o] = l.split() # split in coefficient c and operator string o
            ret.append(PrimitiveOp(SparsePauliOp(o, float(c)))) 
            i += 1
    return ret

def ops_to_circ(ops):
    """Converts list of PrimitiveOp to list of quantum circuits"""
    circs = []
    for o in ops:
        unitary = PauliTrotterEvolution().convert(o.exp_i())
        circs.append(unitary.to_circuit().decompose())
    return circs

def circ_depths(circs):
    """Returns list of depths for transpiled circuits in list circs"""
    depths = []
    for qc in circs:
        qct = transpile(qc, basis_gates=['u','cx'])
        depths.append(qct.depth())
    return depths

qcs = ops_to_circ(ops_from_file("hamiltonian_ordered.txt", 191))
dep = circ_depths(qcs)

In [34]:
print(dep)
sum(dep)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 13, 13, 13, 13, 13, 13, 13, 13, 21, 21, 21, 21, 9, 9, 9, 9, 11, 11, 11, 11, 7, 7, 7, 7, 11, 11, 11, 11, 7, 7, 7, 7, 13, 13, 13, 13, 9, 9, 9, 9, 11, 11, 11, 11, 15, 15, 15, 15, 15, 15, 15, 15, 19, 19, 19, 19, 19, 19, 19, 19, 17, 17, 17, 17, 17, 17, 17, 17, 13, 13, 13, 13, 11, 11, 11, 11, 5, 5, 5, 5, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 9, 9, 9, 9, 13, 13, 13, 13, 15, 15, 15, 15, 11, 11, 11, 11, 15, 15, 15, 15, 11, 11, 11, 11, 9, 9, 9, 9, 11, 11, 11, 11, 9, 9, 9, 9, 7, 7, 7]


1810

In [35]:
# Transpile only at the end
qc = QuantumCircuit(10)
for q in qcs:
    qc.append(q,range(10))
qct = transpile(qc, basis_gates=['u','cx'])
print(qct.depth())

1430


In [ ]:
# Transpile in every step
qc = QuantumCircuit(10)
for q in qcs:
    qc.append(transpile(q, basis_gates=['u','cx']),range(10))
qct = transpile(qc, basis_gates=['u','cx'])
print(qct.depth())